# Heroes of Pymoli Data Analysis
Observed Trend 1: 81% of players purchasing items are male, and the average purchase price for males is $2.95 vs the average purchase price for females at $2.82.
Observed Trend 2: 65.1% of players are between the ages of 15 & 25. These two age groups also makes up 62.7% of total purchases and 62.6% of total revenue.
Observed Trend 3: Three of the most popular items were each purchased 9 times. The top most profitable item was also purchased 9 times. Further analysis could be done to determine if the prices of the most popular items could be marginally increased while still maintining popularity to drive revenue growth. 


In [125]:
import pandas as pd
import os

In [126]:
purchase_data = os.path.join("purchase_data.json")
purchase_data_pd = pd.read_json(purchase_data)

In [127]:
purchase_data_pd.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [128]:
#Player Count (Total Number of Players)
player_count = len(purchase_data_pd["SN"].unique())
player_count_summary = pd.DataFrame({"Total Players": [player_count]})
player_count_summary

,Total Players
0,573


In [129]:
#Purchasing Analysis (Total)
#Number of Unique Items
#Average Purchase Price
#Total Number of Purchases
#Total Revenue

unique_items = len(purchase_data_pd["Item ID"].unique())
average_price = purchase_data_pd["Price"].mean()
total_purchase_count = len(purchase_data_pd)
total_revenue = purchase_data_pd["Price"].sum()

purchasing_analysis = pd.DataFrame({"Number of Unique Items": [unique_items], 
                                    "Average Price": [average_price], 
                                    "Number of Purchases": [total_purchase_count], 
                                    "Total Revenue": [total_revenue]})

purchasing_analysis["Average Price"] = purchasing_analysis["Average Price"].map("${0:,.2f}".format)
purchasing_analysis["Total Revenue"] = purchasing_analysis["Total Revenue"].map("${0:,.2f}".format)

purchasing_analysis = purchasing_analysis[["Number of Unique Items", "Average Price", "Number of Purchases", "Total Revenue"]]
purchasing_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$2.93,780,"$2,286.33"


In [130]:
#Gender Demographics
#Percentage and Count of Male Players
#Percentage and Count of Female Players
#Percentage and Count of Other / Non-Disclosed

player_group_count = purchase_data_pd.groupby(["SN", "Gender"]).count()
player_group_count.reset_index(inplace=True)

total_gender = player_group_count["Gender"].count()
male = player_group_count["Gender"].value_counts()['Male']
female = player_group_count["Gender"].value_counts()['Female']
non_gender_specific = total_gender - male - female
perc_male = (male/total_gender) * 100
perc_female = (female/total_gender) * 100
perc_non_gender_specific = (non_gender_specific/total_gender) * 100

gender_demographic_df = pd.DataFrame({
    "Gender": ["Male", "Female", "Other / Non-Disclosed"],
    "Percentage of Players": [perc_male, perc_female, perc_non_gender_specific],
    "Total Count": [male, female, non_gender_specific]
})
gender_demographic_df["Percentage of Players"] = gender_demographic_df["Percentage of Players"].map("{0:,.2f}".format)
gender_demographic_df = gender_demographic_df.set_index("Gender")
gender_demographic_df

,Percentage of Players,Total Count
Gender,,
Male,81.15,465
Female,17.45,100
Other / Non-Disclosed,1.40,8


In [132]:
#Purchasing Analysis (Gender) 
#The below each broken by gender
#Purchase Count
#Average Purchase Price
#Total Purchase Value
#Normalized Totals

purchasing_gender_group = purchase_data_pd.groupby(['Gender']).agg({'Item ID':'count', 'Price':['mean', 'sum']})

gender_purchasing_df = pd.DataFrame({"Purchase Count": purchasing_gender_group["Item ID"]["count"], 
                                    "Average Purchase Price": purchasing_gender_group["Price"]["mean"], 
                                    "Total Purchase Value": purchasing_gender_group["Price"]["sum"]})

total_gender_counts = pd.DataFrame(gender_demographic_df["Total Count"])
gender_purchasing_df = gender_purchasing_df.join(total_gender_counts)
gender_purchasing_df["Normalized Totals"] = gender_purchasing_df["Total Purchase Value"] / gender_purchasing_df["Total Count"]
gender_purchasing_df = gender_purchasing_df.drop("Total Count", axis=1)

gender_purchasing_df=gender_purchasing_df[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]
gender_purchasing_df["Average Purchase Price"] = gender_purchasing_df["Average Purchase Price"].map("${0:,.2f}".format)
gender_purchasing_df["Total Purchase Value"] = gender_purchasing_df["Total Purchase Value"].map("${0:,.2f}".format)
gender_purchasing_df["Normalized Totals"] = gender_purchasing_df["Normalized Totals"].map("${0:,.2f}".format)
gender_purchasing_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,136,$2.82,$382.91,$3.83
Male,633,$2.95,"$1,867.68",$4.02
Other / Non-Disclosed,11,$3.25,$35.74,$4.47


In [133]:
#Age Demographics
bins = [0,10,15,20,25,30,35,40,100]
group_labels = ["<10","10-14","15-19","20-24","25-29","30-34",
                "35-39","40+"]

unique_screenname = purchase_data_pd.groupby(['SN']).max()

player_age_df = unique_screenname.groupby(pd.cut(unique_screenname["Age"], bins, labels=group_labels))
player_age_df = pd.DataFrame(player_age_df.size())
player_age_df.columns = ["Total Count"] 
player_age_df["Percentage of Players"] = player_age_df["Total Count"] / player_age_df["Total Count"].sum() * 100        
player_age_df["Percentage of Players"] = player_age_df["Percentage of Players"].map("{0:,.2f}".format)  
player_age_df = player_age_df[["Percentage of Players", "Total Count"]]
player_age_df


,Percentage of Players,Total Count
Age,,
<10,3.84,22
10-14,9.42,54
15-19,24.26,139
20-24,40.84,234
25-29,9.08,52
30-34,7.68,44
35-39,4.36,25
40+,0.52,3


In [134]:
#Age Demographics
#The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.) 
#Purchase Count
#Average Purchase Price
#Total Purchase Value
#Normalized Totals

bins = [0,10,15,20,25,30,35,40,100]
group_labels = ["<10","10-14","15-19","20-24","25-29","30-34",
                "35-39","40+"]

age_purchase_group = purchase_data_pd.groupby(pd.cut(purchase_data_pd["Age"], bins, labels=group_labels))
age_purchase_group = age_purchase_group.agg({'Item ID':'count', 'Price':['mean', 'sum']})

age_summary = pd.DataFrame({"Purchase Count": age_purchase_group["Item ID"]["count"], 
                                    "Average Purchase Price": age_purchase_group["Price"]["mean"], 
                                    "Total Purchase Value": age_purchase_group["Price"]["sum"]})

total_age_counts = pd.DataFrame(player_age_df["Total Count"])
age_summary = age_summary.join(total_age_counts)
age_summary["Normalized Totals"] = age_summary["Total Purchase Value"] / age_summary["Total Count"]
age_summary = age_summary.drop("Total Count", axis=1)

age_summary=age_summary[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]
age_summary["Average Purchase Price"] = age_summary["Average Purchase Price"].map("${0:,.2f}".format)
age_summary["Total Purchase Value"] = age_summary["Total Purchase Value"].map("${0:,.2f}".format)
age_summary["Normalized Totals"] = age_summary["Normalized Totals"].map("${0:,.2f}".format)
age_summary

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age,,,,
<10,32,$3.02,$96.62,$4.39
10-14,78,$2.87,$224.15,$4.15
15-19,184,$2.87,$528.74,$3.80
20-24,305,$2.96,$902.61,$3.86
25-29,76,$2.89,$219.82,$4.23
30-34,58,$3.07,$178.26,$4.05
35-39,44,$2.90,$127.49,$5.10
40+,3,$2.88,$8.64,$2.88


In [135]:
#Top Spenders
#Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
#SN
#Purchase Count
#Average Purchase Price
#Total Purchase Value

top_spenders = purchase_data_pd.groupby(['SN'])
top_spenders_group = top_spenders.agg({'Item ID':'count', 'Price':['mean', 'sum']})

top_spenders_group = pd.DataFrame({"Purchase Count": top_spenders_group["Item ID"]["count"], 
                                    "Average Purchase Price": top_spenders_group["Price"]["mean"], 
                                    "Total Purchase Value": top_spenders_group["Price"]["sum"]})
top_spenders_group=top_spenders_group[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]
top_spenders_group["Average Purchase Price"] = top_spenders_group["Average Purchase Price"].map("${0:,.2f}".format)
top_spenders_group["Total Purchase Value"] = top_spenders_group["Total Purchase Value"].map("${0:,.2f}".format)

top_spenders_group = top_spenders_group.sort_values(["Total Purchase Value"], ascending=False)
top_spenders_group.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Qarwen67,4,$2.49,$9.97
Sondim43,3,$3.13,$9.38
Tillyrin30,3,$3.06,$9.19
Lisistaya47,3,$3.06,$9.19
Tyisriphos58,2,$4.59,$9.18


In [136]:
#Most Popular Items
#Identify the 5 most popular items by purchase count, then list (in a table):
#Item ID
#Item Name
#Purchase Count
#Item Price
#Total Purchase Value

popular_items = purchase_data_pd.groupby(['Item ID'])
popular_items_group = popular_items.agg({'Item Name':'max', 'SN':'count','Price':['max', 'sum']})

popular_items_group = pd.DataFrame({"Item Name": popular_items_group["Item Name"]["max"],
                                    "Purchase Count": popular_items_group["SN"]["count"], 
                                    "Item Price": popular_items_group["Price"]["max"], 
                                    "Total Purchase Value": popular_items_group["Price"]["sum"]})
popular_items_group=popular_items_group[["Item Name","Purchase Count", "Item Price", "Total Purchase Value"]]
popular_items_group = popular_items_group.sort_values(["Purchase Count"], ascending=False)
popular_items_group["Item Price"] = popular_items_group["Item Price"].map("${0:,.2f}".format)
popular_items_group["Total Purchase Value"] = popular_items_group["Total Purchase Value"].map("${0:,.2f}".format)

popular_items_group.head(5)

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
31,Trickster,9,$2.07,$18.63
175,Woeful Adamantite Claymore,9,$1.24,$11.16
13,Serenity,9,$1.49,$13.41


In [137]:
#Most Profitable Items
#Identify the 5 most profitable items by total purchase value, then list (in a table):
#Item ID
#Item Name
#Purchase Count
#Item Price
#Total Purchase Value

profit_items = purchase_data_pd.groupby(['Item ID'])
profit_items_group = profit_items.agg({'Item Name':'max', 'SN':'count','Price':['max', 'sum']})

profit_items_group = pd.DataFrame({"Item Name": profit_items_group["Item Name"]["max"],
                                    "Purchase Count": profit_items_group["SN"]["count"], 
                                    "Item Price": profit_items_group["Price"]["max"], 
                                    "Total Purchase Value": profit_items_group["Price"]["sum"]})
profit_items_group=profit_items_group[["Item Name","Purchase Count", "Item Price", "Total Purchase Value"]]
profit_items_group = profit_items_group.sort_values(["Total Purchase Value"], ascending=False)
profit_items_group["Item Price"] = profit_items_group["Item Price"].map("${0:,.2f}".format)
profit_items_group["Total Purchase Value"] = profit_items_group["Total Purchase Value"].map("${0:,.2f}".format)

profit_items_group.head(5)

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88
